In [1]:
import numpy as np
import pennylane as qml
from qiskit.quantum_info import SparsePauliOp, Pauli
from susy_qm import calculate_Hamiltonian, calculate_Hamiltonian2

In [5]:
cutoff = 2
potential = 'DW'
H = calculate_Hamiltonian(cutoff, potential)

In [6]:
SparsePauliOp.from_operator(H)

SparsePauliOp(['II', 'IX', 'ZI', 'ZX'],
              coeffs=[1.625     +0.j, 1.06066017+0.j, 0.5       +0.j, 0.70710678+0.j])

In [7]:
qml.pauli_decompose(H)

(
    1.625 * (I(0) @ I(1))
  + 1.0606601717798212 * (I(0) @ X(1))
  + 0.5 * (Z(0) @ I(1))
  + 0.7071067811865475 * (Z(0) @ X(1))
)

In [18]:
def pennylane_to_qiskit_sparse_pauliop(pl_hamiltonian, wire_order=None):
    """Convert a PennyLane Hamiltonian into a Qiskit SparsePauliOp with full precision."""
    if wire_order is None:
        wire_order = sorted(pl_hamiltonian.wires.tolist())

    pauli_labels = []
    coeffs = []

    for coeff, op in zip(pl_hamiltonian.coeffs, pl_hamiltonian.ops):
        label = ['I'] * len(wire_order)

        # Handle tensor products
        ops = [op] if not hasattr(op, "obs") else op.obs
        for subop in ops:
            wire = subop.wires[0]
            wire_index = wire_order.index(wire)
            label[wire_index] = subop.name[-1].upper()  # ensure uppercase

        pauli_labels.append("".join(label))
        coeffs.append(np.complex128(coeff))  # force high precision

    return SparsePauliOp.from_list(list(zip(pauli_labels, coeffs)), dtype=np.complex128)

In [20]:
pennylane_to_qiskit_sparse_pauliop(qml.pauli_decompose(H))

SparsePauliOp(['YYY', 'YXY', 'YXZ', 'YZY', 'YZZ', 'ZYY', 'ZXY', 'ZXZ', 'ZZY', 'ZZZ'],
              coeffs=[10.3125    +0.j,  6.88222151+0.j, -1.8440857 +0.j, -4.0625    +0.j,
 -4.0625    +0.j,  2.75      +0.j,  1.44888874+0.j, -0.38822857+0.j,
 -0.75      +0.j, -0.75      +0.j])

In [2]:
cutoff = 4
potential = 'DW'
m=1
g=1
u=1

In [3]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential, m, g, u)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]

In [4]:
eigenvalues.real

array([0.90655987, 0.95063353, 1.69566635, 1.83552815])

In [5]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian2(cutoff, potential, m, g, u)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]

In [6]:
eigenvalues.real

array([0.90655987, 0.95063353, 1.69566635, 1.83552815])